In [2]:
import requests
import json

def fetch_countries_data():
    # Using session is particularly beneficial if you are making multiple requests to the same server, 
    # as it can reuse the underlying TCP connection, 
    # leading to performance improvements.
    with requests.Session() as session:
        url = "https://restcountries.com/v3.1/all"
        response = session.get(url)
        response.raise_for_status()
        
        if response.status_code == 200:
            return response.json()
        else:
            return f"Error: {response.status_code}"

# Fetch and print data
countries_data = fetch_countries_data()
# print(type(countries_data))


In [6]:
import os
os.environ["MINIO_KEY"] = "minio"
os.environ["MINIO_SECRET"] = "minio123"

In [12]:
!pip install minio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 2.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.8 MB/s eta 0:00:0000:0100:01m


In [13]:
!pip install s3fs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 978.2 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.1/328.1 kB 1.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.9.2
    Uninstalling fsspec-2023.9.2:
      Successfully uninstalled fsspec-2023.9.2


In [7]:
from minio import Minio
import pandas as pd
import s3fs

fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'http://minio1:9000'},
    key=os.environ["MINIO_KEY"],
    secret=os.environ["MINIO_SECRET"],
    use_ssl=False  # Set to True if MinIO is set up with SSL
)

In [7]:
with fs.open('mybucket/data.json', 'w', encoding='utf-8') as f:
    json.dump(countries_data,f)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("MinIO Test") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio1:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .enableHiveSupport() \
    .getOrCreate()

In [10]:
df = spark.read.option("inferSchema",True).json("s3a://mybucket/data.json")

In [15]:
# df.printSchema()

In [12]:
df.count()

250

In [14]:
# df.show(2,truncate = False)

In [16]:
df.write.mode("overwrite").format("parquet").save("s3a://mybucket/raw_data.parquet")

In [2]:
raw_data = spark.read.parquet("s3a://mybucket/raw_data.parquet")
raw_data.count()

250

In [9]:
transformed_data = (
    raw_data.selectExpr(
        "name.common as cntry_name",
        "area as cntry_area",
        "borders as border_cntry",
        "capital as capital_cities",
        "continents as cntry_continent",
        "landlocked as is_landlocked",
        "population",
        "startOfWeek",
        "timezones as nr_timezones",
        "unMember as is_unmember"
    )
)
transformed_data.printSchema()

root
 |-- cntry_name: string (nullable = true)
 |-- cntry_area: double (nullable = true)
 |-- border_cntry: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- capital_cities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cntry_continent: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_landlocked: boolean (nullable = true)
 |-- population: long (nullable = true)
 |-- startOfWeek: string (nullable = true)
 |-- nr_timezones: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_unmember: boolean (nullable = true)



In [10]:
transformed_data.show(10,truncate = False)

+----------------+----------+-------------------------+------------------+---------------+-------------+----------+-----------+---------------------------------+-----------+
|cntry_name      |cntry_area|border_cntry             |capital_cities    |cntry_continent|is_landlocked|population|startOfWeek|nr_timezones                     |is_unmember|
+----------------+----------+-------------------------+------------------+---------------+-------------+----------+-----------+---------------------------------+-----------+
|Christmas Island|135.0     |NULL                     |[Flying Fish Cove]|[Asia]         |false        |2072      |monday     |[UTC+07:00]                      |false      |
|Eritrea         |117600.0  |[DJI, ETH, SDN]          |[Asmara]          |[Africa]       |false        |5352000   |monday     |[UTC+03:00]                      |true       |
|Samoa           |2842.0    |NULL                     |[Apia]            |[Oceania]      |false        |198410    |monday     |[UT

In [11]:
transformed_data.write.mode("overwrite").format("parquet").save("s3a://mybucket/trnsfm_data.parquet")

In [12]:
spark.sql("""
CREATE EXTERNAL TABLE country_data (
    cntry_name STRING,
    cntry_area DOUBLE,
    border_cntry ARRAY<STRING>,
    capital_cities ARRAY<STRING>,
    cntry_continent ARRAY<STRING>,
    is_landlocked BOOLEAN,
    population BIGINT,
    startOfWeek STRING,
    nr_timezones ARRAY<STRING>,
    is_unmember BOOLEAN
)
STORED AS PARQUET
LOCATION 's3a://mybucket/trnsfm_data.parquet';
""").show()

++
||
++
++



In [13]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [18]:
spark.sql("DESCRIBE EXTENDED default.country_data").show(100,truncate = False)

+----------------------------+--------------------------------------------------------------+-------+
|col_name                    |data_type                                                     |comment|
+----------------------------+--------------------------------------------------------------+-------+
|cntry_name                  |string                                                        |NULL   |
|cntry_area                  |double                                                        |NULL   |
|border_cntry                |array<string>                                                 |NULL   |
|capital_cities              |array<string>                                                 |NULL   |
|cntry_continent             |array<string>                                                 |NULL   |
|is_landlocked               |boolean                                                       |NULL   |
|population                  |bigint                                              

In [21]:
# raw_data.printSchema()

In [9]:
cd = spark.table("default.country_data").toPandas()
with fs.open('mybucket/country_data.csv', 'w') as f:
    cd.to_csv(f, index=False, header = True)
# cd.to_csv("s3a://mybucket/country_data.csv",index = False, header = True)

In [18]:
# preprocessing
country_data = spark.table("default.country_data")
country_data_processed = (
    country_data
    .withColumn("cntry_area",F.when(F.col("cntry_area") < 0, None).otherwise(F.col("cntry_area")))
    .withColumn("border_cntry",F.when(F.col("border_cntry").isNull(),F.array(F.lit("NA"))).otherwise(F.col("border_cntry")))
    .withColumn("capital_cities",F.when(F.col("capital_cities").isNull(),F.array(F.lit("NA"))).otherwise(F.col("capital_cities")))
)
country_data_processed.printSchema()

root
 |-- cntry_name: string (nullable = true)
 |-- cntry_area: double (nullable = true)
 |-- border_cntry: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- capital_cities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cntry_continent: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_landlocked: boolean (nullable = true)
 |-- population: long (nullable = true)
 |-- startOfWeek: string (nullable = true)
 |-- nr_timezones: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_unmember: boolean (nullable = true)



In [10]:
spark.sql("select * from default.country_data where cntry_name = 'French Polynesia'").show(truncate = False)

+----------------+----------+------------+--------------+---------------+-------------+----------+-----------+---------------------------------+-----------+
|cntry_name      |cntry_area|border_cntry|capital_cities|cntry_continent|is_landlocked|population|startOfWeek|nr_timezones                     |is_unmember|
+----------------+----------+------------+--------------+---------------+-------------+----------+-----------+---------------------------------+-----------+
|French Polynesia|4167.0    |NULL        |[Papeetē]     |[Oceania]      |false        |280904    |monday     |[UTC-10:00, UTC-09:30, UTC-09:00]|false      |
+----------------+----------+------------+--------------+---------------+-------------+----------+-----------+---------------------------------+-----------+



In [44]:
country_data_processed.filter("cntry_name = 'Slovakia'").show(truncate = False)

+----------+----------+-------------------------+--------------+---------------+-------------+----------+-----------+------------+-----------+
|cntry_name|cntry_area|border_cntry             |capital_cities|cntry_continent|is_landlocked|population|startOfWeek|nr_timezones|is_unmember|
+----------+----------+-------------------------+--------------+---------------+-------------+----------+-----------+------------+-----------+
|Slovakia  |49037.0   |[AUT, CZE, HUN, POL, UKR]|[Bratislava]  |[Europe]       |true         |5458827   |monday     |[UTC+01:00] |true       |
+----------+----------+-------------------------+--------------+---------------+-------------+----------+-----------+------------+-----------+



In [20]:
country_data_processed.createOrReplaceTempView("country_data_processed_view")

In [21]:
spark.sql("DESCRIBE EXTENDED country_data_processed_view").show(truncate = False)

+---------------+-------------+-------+
|col_name       |data_type    |comment|
+---------------+-------------+-------+
|cntry_name     |string       |NULL   |
|cntry_area     |double       |NULL   |
|border_cntry   |array<string>|NULL   |
|capital_cities |array<string>|NULL   |
|cntry_continent|array<string>|NULL   |
|is_landlocked  |boolean      |NULL   |
|population     |bigint       |NULL   |
|startOfWeek    |string       |NULL   |
|nr_timezones   |array<string>|NULL   |
|is_unmember    |boolean      |NULL   |
+---------------+-------------+-------+



In [23]:
def show_results(sql_string):
    return spark.sql(
        sql_string
    ).show(truncate = False)

In [24]:
# 1. Which are the 10 largest countries in terms of area? (in sq. km.)
sql_string = """
    SELECT cntry_name, cntry_area
    FROM country_data_processed_view
    ORDER BY cntry_area DESC
    LIMIT 10
    """
show_results(sql_string)

+-------------+-----------+
|cntry_name   |cntry_area |
+-------------+-----------+
|Russia       |1.7098242E7|
|Antarctica   |1.4E7      |
|Canada       |9984670.0  |
|China        |9706961.0  |
|United States|9372610.0  |
|Brazil       |8515767.0  |
|Australia    |7692024.0  |
|India        |3287590.0  |
|Argentina    |2780400.0  |
|Kazakhstan   |2724900.0  |
+-------------+-----------+



In [31]:
# 2. Which country has the largest number of neighbouring countries?
sql_string = """
    SELECT cntry_name, border_cntry, array_size(border_cntry) as ngbr_cntry_nr
    FROM country_data_processed_view
    WHERE NOT array_contains(border_cntry,'NA')
    ORDER BY array_size(border_cntry) DESC
    LIMIT 1
    """
show_results(sql_string)

+----------+--------------------------------------------------------------------------------+-------------+
|cntry_name|border_cntry                                                                    |ngbr_cntry_nr|
+----------+--------------------------------------------------------------------------------+-------------+
|China     |[AFG, BTN, MMR, HKG, IND, KAZ, NPL, PRK, KGZ, LAO, MAC, MNG, PAK, RUS, TJK, VNM]|16           |
+----------+--------------------------------------------------------------------------------+-------------+



In [33]:
# 3. Which countries have the highest number of capital cities?
sql_string = """
    SELECT cntry_name, capital_cities, array_size(capital_cities) as total_capital_cities
    FROM country_data_processed_view
    WHERE NOT array_contains(capital_cities,'NA')
    ORDER BY array_size(capital_cities) DESC
    LIMIT 2
    """
show_results(sql_string)

+------------+-----------------------------------+--------------------+
|cntry_name  |capital_cities                     |total_capital_cities|
+------------+-----------------------------------+--------------------+
|South Africa|[Pretoria, Bloemfontein, Cape Town]|3                   |
|Palestine   |[Ramallah, Jerusalem]              |2                   |
+------------+-----------------------------------+--------------------+



In [35]:
# 4. How many countries lie on two or more continents?
sql_string = """
    SELECT cntry_name, cntry_continent, array_size(cntry_continent) as total_continents
    FROM country_data_processed_view
    ORDER BY array_size(cntry_continent) DESC
    LIMIT 3
    """
show_results(sql_string)

+----------+---------------+----------------+
|cntry_name|cntry_continent|total_continents|
+----------+---------------+----------------+
|Turkey    |[Europe, Asia] |2               |
|Azerbaijan|[Europe, Asia] |2               |
|Russia    |[Europe, Asia] |2               |
+----------+---------------+----------------+



In [45]:
# 5. How many landlocked countries per continent?
sql_string = """
    SELECT continent, SUM(is_landlocked) as landlocked_nr
    FROM (SELECT cntry_name, case when is_landlocked then 1 else 0 end as is_landlocked, explode(cntry_continent) as continent
    FROM country_data_processed_view)
    GROUP BY continent
    ORDER BY SUM(is_landlocked) DESC
    """
show_results(sql_string)

+-------------+-------------+
|continent    |landlocked_nr|
+-------------+-------------+
|Europe       |16           |
|Africa       |16           |
|Asia         |12           |
|South America|2            |
|North America|0            |
|Antarctica   |0            |
|Oceania      |0            |
+-------------+-------------+



In [47]:
# 6. Which country has the highest number of time zones?
sql_string = """
    SELECT cntry_name, nr_timezones, array_size(nr_timezones) as total_timezones
    FROM country_data_processed_view
    ORDER BY array_size(nr_timezones) DESC
    LIMIT 1
    """
show_results(sql_string)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|cntry_name|nr_timezones                                                                                                                                              |total_timezones|
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|France    |[UTC-10:00, UTC-09:30, UTC-09:00, UTC-08:00, UTC-04:00, UTC-03:00, UTC+01:00, UTC+02:00, UTC+03:00, UTC+04:00, UTC+05:00, UTC+10:00, UTC+11:00, UTC+12:00]|14             |
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+



In [52]:
# 7. How many countries are not UN members?
sql_string = """
    SELECT COUNT(*) AS count
    FROM country_data_processed_view
    WHERE NOT is_unmember
    """
show_results(sql_string)

+-----+
|count|
+-----+
|57   |
+-----+



In [8]:
# import os

# # Retrieve all environment variables
# environment_variables = os.environ

# # Print each environment variable
# for key, value in environment_variables.items():
#     print(f"{key}: {value}")